In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from xgboost.sklearn import XGBClassifier

Populated interactive namespace

In [2]:
xtrain=pd.read_csv('train/X_train.csv', sep=' ', header=None)
ytrain=pd.read_csv('train/Y_train.csv', header=None)

xtrain.shape

(7767, 561)

In [3]:
ytrain.shape

(7767, 1)

Read in CSV files

In [4]:
xtest=pd.read_csv('test/X_test.csv', sep=' ', header=None)
ytest=pd.read_csv('test/Y_test.csv', header=None)
xtest.shape
ytest.shape

(3162, 1)

In [5]:
tree=XGBClassifier(objective = 'multiclass:softmax' , learning_rate=0.1 ,n_estimators=1000)

tree.fit(xtrain, ytrain)


pred_tree =  tree.predict(xtest)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Created predicteds, now to test against ytest.

In [6]:
print(accuracy_score(ytest,pred_tree))

0.9380139152435167


Accuracy of 93.8% with non-optimized params

In [12]:
#test cv params for best params, optimizing

cv_params = {'n_estimators':[500,1000],'max_depth':[3,5]}
clf_params={'objective':'multiclass:softmax','nthread':3,'max_depth':3,'n_estimators':1000,'subsample':0.8}

clf=XGBClassifier(**clf_params)

model= GridSearchCV(clf, cv_params, scoring='accuracy', cv=3, n_jobs=3, return_train_score=True)

In [13]:
#now run our gridsearch with cross validation to see which params perform best.
#should take a LONG time
model.fit(xtrain, ytrain)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth={'objective': 'multiclass:softmax', 'nthread': 3, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 0.8},
       min_child_weight=...0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=3,
       param_grid={'n_estimators': [500, 1000], 'max_depth': [3, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [15]:
#we use grid scores to determine best params.
model.cv_results_


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\util

{'mean_fit_time': array([306.88682055, 467.14427916, 304.69010941, 462.81661495]),
 'std_fit_time': array([4.4638369 , 6.33198796, 3.60128608, 3.78786836]),
 'mean_score_time': array([0.36441628, 0.55822929, 0.35139481, 0.65872653]),
 'std_score_time': array([0.00484944, 0.0102676 , 0.00839223, 0.13047217]),
 'param_max_depth': masked_array(data=[3, 3, 5, 5],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[500, 1000, 500, 1000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 3, 'n_estimators': 500},
  {'max_depth': 3, 'n_estimators': 1000},
  {'max_depth': 5, 'n_estimators': 500},
  {'max_depth': 5, 'n_estimators': 1000}],
 'split0_test_score': array([0.93245851, 0.93130066, 0.92551139, 0.92589734]),
 'split1_test_score': array([0.86949807, 0.87104247, 0.86486486, 0.86447876]),
 'split2_test_score': array([0.92962

In [16]:
model.best_params_

{'max_depth': 3, 'n_estimators': 1000}

In [17]:
model.best_score_

0.9106476116904854

The best parameters to use are max_depth: 3 and n_estimators:1000. Now we will optimize subsampling and learning rate.

In [23]:
cv_params2={'learning_rate': [0.1,0.01], 'subsample': [0.5,1]}
clf_params2={'objective':'multiclass:softmax','nthread':3,'max_depth':3,'n_estimators':1000}

clf2=XGBClassifier(**clf_params2)
model2=GridSearchCV(clf2, cv_params2, scoring='accuracy', cv=3, n_jobs=3, return_train_score=True)

In [24]:
model2.fit(xtrain,ytrain)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=3, objective='multiclass:softmax', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=3,
       param_grid={'learning_rate': [0.1, 0.01], 'subsample': [0.5, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [26]:
model2.cv_results_, model2.best_params_, model2.best_score_

({'mean_fit_time': array([354.1933864 , 349.51567912, 687.66467889, 841.14352083]),
  'std_fit_time': array([4.31256926, 3.62213255, 1.87877431, 5.6422966 ]),
  'mean_score_time': array([0.27483241, 0.27953744, 0.55651283, 0.92818228]),
  'std_score_time': array([0.01255471, 0.0082492 , 0.01894041, 0.4165533 ]),
  'param_learning_rate': masked_array(data=[0.1, 0.1, 0.01, 0.01],
               mask=[False, False, False, False],
         fill_value='?',
              dtype=object),
  'param_subsample': masked_array(data=[0.5, 1, 0.5, 1],
               mask=[False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'learning_rate': 0.1, 'subsample': 0.5},
   {'learning_rate': 0.1, 'subsample': 1},
   {'learning_rate': 0.01, 'subsample': 0.5},
   {'learning_rate': 0.01, 'subsample': 1}],
  'split0_test_score': array([0.93477422, 0.93130066, 0.92821305, 0.92396758]),
  'split1_test_score': array([0.89305019, 0.87104247, 0.87451737, 0.86602317]),
  'sp

Our best params are now learning_rate:0.1, subsample: 0.5, 'max_depth': 3, 'n_estimators': 1000. We can further tune our parameters but this is enough to get relatively high accuracy. Now we compare tuned model to non-optimized model.

In [29]:
final_tree=XGBClassifier(objective = 'multiclass:softmax' , learning_rate=0.1 ,n_estimators=1000, subsample=0.5, max_depth=3, nthreads=3, n_jobs=3)

final_tree.fit(xtrain,ytrain)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=3, nthread=None, nthreads=3, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.5)

In [30]:
final_pred=final_tree.predict(xtest)
print(accuracy_score(ytest,final_pred))

0.9421252371916509


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


As we can see when optimizing parameters, we get a higher accuracy score in our predicted activity.

In [32]:
final_pred=pd.DataFrame(final_pred)

In [33]:
final_pred.head()

,0
0,5
1,5
2,5
3,5
4,5


In [37]:
final_pred=pd.concat([final_pred, xtest], axis=1)

In [38]:
final_pred=pd.concat([final_pred,ytest],axis=1)

In [40]:
final_pred.to_csv('Final Predicteds with Xtest data vs. Ytest data.csv')

Saved data to view as csv.